In [1]:
using Revise

In [2]:
using ModelVerification

┌ Warning: Error requiring `Flux` from `ModelVerification`
│   exception = (ErrorException("cannot assign a value to variable Flux.Flux from module ModelVerification"), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007ffaed966692, Ptr{Nothing} @0x00007ffaed996e3f, Ptr{Nothing} @0x00007ffaed9bdc46, Ptr{Nothing} @0x00007ffaed99b77a, Ptr{Nothing} @0x00007ffaed99c411, Base.InterpreterIP in top-level CodeInfo for ModelVerification at statement 0, Ptr{Nothing} @0x00007ffaed9bcdca, Ptr{Nothing} @0x00007ffaed9bec2a, Ptr{Nothing} @0x00007ff85ff0eb3d, Ptr{Nothing} @0x00007ff85ff0eb5f, Ptr{Nothing} @0x00007ffaed97ddbd, Ptr{Nothing} @0x00007ffaed99b084, Ptr{Nothing} @0x00007ffaed99aac7, Ptr{Nothing} @0x00007ffaed99b9aa, Ptr{Nothing} @0x00007ffaed99bac7, Ptr{Nothing} @0x00007ffaed99c097, Base.InterpreterIP in MethodInstance for Requires.err(::Any, ::Module, ::String, ::String, ::Any) at statement 4, Ptr{Nothing} @0x00007ff85ff0eaa0, Ptr{Nothing} @0x00007ff85ff0eabf, Ptr{Nothing} @0x0000

In [3]:
using LazySets
using PyCall
using CSV
using ONNX
using Flux
using Test
using NNlib
using ONNXNaiveNASflux
using NaiveNASflux
# using DataFrames
# import Flux: flatten

In [4]:
# using Flux: onehotbatch, onecold, flatten
# using Flux.Losses: logitcrossentropy
# using Statistics: mean
using CUDA
using MLDatasets: CIFAR10, MNIST
using MLUtils: splitobs, DataLoader
using Accessors
using Profile
using LinearAlgebra
using Einsum

In [5]:
struct A
    index
    name
    alpha
end
function (f::A)(x)
    return f.alpha .* x
end
Flux.@functor A (alpha,)


a = A(1, "a", [1.0])
b = A(2, "b", [2.0])
c = A(3, "c", [3.0])
d = A(4, "d", [4.0])
e = A(5, "e", [5.0])
f(x) = 1 .+ x
m = [a, Chain([b, f, Chain([c, d])]), e, f]
model = Chain(m)
println(model)
train_set = [[1], [2]]
println(Flux.params(model))
optimizer = Flux.Optimiser(Flux.ADAM(0.1))
my_loss(x, y) = sum((x .- y) .^ 2)
state = Flux.setup(optimizer, model)
for i in 1 : 10
    value, grads = Flux.withgradient(model) do m
        result = m(train_set[1])
        my_loss(result, train_set[2])
    end
    Flux.update!(state, model, grads[1])
    println(Flux.params(model))
end

    

Chain([

A(1, 

"a", 

[1.0]), Chain([A(2, "b", [2.0]), f

, Chain([A(3, "c", [3.0]), A(4, "d", [4.0])])]), A(5, "e", [5.0]), f])
Params([

[1.0], [2.0], [3.0], [4.0], [5.0]])
Params([

[0.9000000000000232], [1.9000000000000465], [2.9000000000000465], [3.900000000000062], [4.900000000000078]])
Params([[0.8019825932831908], [1.8025378886634913], [2.8021459853532504], [3.802233177187348], [4.802285785675325]])
Params([[0.7071285537155807], [1.7091328179962304], [2.707686153417145], [3.7079906353293053], [4.708173053538199]])
Params([[0.6163592164651227], [1.620926698255525], [2.6175636418606336], [3.6182364246944028], [4.618636539760596]])
Params([[0.530297047352178], [1.538623508664359], [2.532384228495501], [3.533576511686647], [4.534280370716337]])
Params([[0.4492866064007697], [1.46253129652812], [2.452451856611642], [3.4543004667784043], [4.455383951166571]])
Params([[0.37344761909996416], [1.3926549079493182], [2.377835846283726], [3.3804556101025662], [4.38198048756879]])
Params([[0.3027355249790223], [1.3287951106191929], [2.3084414050913495], [3.311922903165741], [4.3139360880194495]])
Params([[0.23699541870781865], [1.2706311206777086], [2.244069037459118], [3

In [11]:
struct AlphaLayer
    node
    alpha
    lower
    unstable_mask
    lower_mask 
    upper_slope
    lower_bias
    upper_bias
end
Flux.@functor AlphaLayer (alpha,)

function (f::AlphaLayer)(x)
    Last_A = x[1]
    Last_bias = x[2]
    lower_slope = clamp.(f.alpha, 0, 1) .* f.unstable_mask .+ f.lower_mask 
    if f.lower 
        New_A = bound_oneside(Last_A, lower_slope, f.upper_slope)
    else
        New_A = bound_oneside(Last_A, f.upper_slope, lower_slope)
    end

    if isnothing(Last_bias)
        return [New_A, nothing]
    end
    New_bias = multiply_bias(Last_bias, f.upper_slope, f.upper_bias, f.lower_bias)

    return [New_A, New_bias]
end

#Upper bound slope and intercept according to CROWN relaxation.
function relu_upper_bound(lower, upper)
    lower_r = clamp.(lower, -Inf, 0)
    upper_r = clamp.(upper, 0, Inf)
    upper_r .= max.(upper_r, lower_r .+ 1e-8)
    upper_slope = upper_r ./ (upper_r .- lower_r) #the slope of the relu upper bound
    upper_bias = - lower_r .* upper_slope #the bias of the relu upper bound
    return upper_slope, upper_bias
end

function clamp_mutiply_A(last_A, slope_pos, slope_neg) 
    A_pos = clamp.(last_A, 0, Inf)
    A_neg = clamp.(last_A, -Inf, 0)
    slope_pos = repeat(reshape(slope_pos,(1, size(slope_pos)...)), size(A_pos)[1], 1, 1) #add spec dim for slope_pos
    slope_neg = repeat(reshape(slope_neg,(1, size(slope_neg)...)), size(A_neg)[1], 1, 1) #add spec dim for slope_pos
    New_A = slope_pos .* A_pos .+ slope_neg .* A_neg 
    return New_A
end 


function clamp_mutiply_bias(last_A, bias_pos, bias_neg) 
    A_pos = clamp.(last_A, 0, Inf)
    A_neg = clamp.(last_A, -Inf, 0) 
    if bias_pos !== nothing #new_bias_pos = torch.einsum('s...b,s...b->sb', A_pos, bias_pos)
        new_bias_pos = zeros((size(A_pos)[1], size(A_pos)[end]))#spec_dim x batch dim
        @einsum new_bias_pos[s,b] = A_pos[s,r,b] * bias_pos[r,b]
    end

    if bias_neg !== nothing #new_bias_neg = torch.einsum('...sb,...sb->sb', A_neg, bias_neg)
        new_bias_neg = zeros((size(A_neg)[1], size(A_neg)[end]))#spec_dim x batch dim
        @einsum new_bias_neg[s,b] = A_neg[s,r,b] * bias_neg[r,b]
    end
    New_bias = new_bias_pos .+ new_bias_neg
    return New_bias
end 

#using last_A for getting New_A
function multiply_by_A_signs(last_A, slope_pos, slope_neg)
    if ndims(slope_pos) == 1
        # Special case for LSTM, the bias term is 1-dimension. 
        New_A = clamp.(last_A, 0, Inf) .* slope_pos .+ clamp.(last_A, -Inf, 0) .* slope_neg
    else
        New_A = clamp_mutiply_A(last_A, slope_pos, slope_neg)
        return New_A
    end
end

function multiply_bias(last_A, upper_slope, bias_pos, bias_neg)
    if ndims(upper_slope) == 1
        # Special case for LSTM, the bias term is 1-dimension. 
        New_bias = clamp.(last_A, 0, Inf) .* bias_pos .+ clamp.(last_A, -Inf, 0) .* bias_neg
    else
        New_bias = clamp_mutiply_bias(last_A, bias_pos, bias_neg)
        return New_bias
    end
end

#bound oneside of the relu, like upper or lower
function bound_oneside(last_A, slope_pos, slope_neg)
    if isnothing(last_A)
        return nothing, nothing
    end
    New_A = multiply_by_A_signs(last_A, slope_pos, slope_neg)
    return New_A
end


alpha_lower = [20]
alpha_upper = [1]
unstable_mask = [1]
lower_mask = [1]
upper_slope = [2]
upper_bias = [0]
lower_bias = [0]
lower = upper = true
if lower == true
    Alpha_Lower_Layer = AlphaLayer("relu_1", alpha_lower, true, unstable_mask, lower_mask, upper_slope, upper_bias, lower_bias)
end
if upper ==true
    Alpha_Upper_Layer = AlphaLayer("relu_1", alpha_upper, false, unstable_mask, lower_mask, upper_slope, lower_bias, upper_bias)
end
a = []
push!(a, Alpha_Lower_Layer) 
push!(a, Alpha_Upper_Layer)
println(Flux.params(Alpha_Lower_Layer))
println(Flux.params(a)) 
a = Chain(a)
println(a([2, 0]))

Params([[20]])
Params([[20], [1]])


[[4.0], [0.0]]


[[32.0], [0.0]]


In [9]:
struct AlphaLayer
    alpha
    lower
    unstable_mask
    lower_mask 
    upper_slope
end
Flux.@functor AlphaLayer (alpha,)

function (l::AlphaLayer)(x)
    lb_lower_slope = clamp.(l.alpha, 0, 1) .* l.unstable_mask .+ l.lower_mask 
    if l.lower
        New_A = bound_oneside(x, lb_lower_slope, l.upper_slope)
    else
        New_A = [-1]
    end
    return New_A
end


#Upper bound slope and intercept according to CROWN relaxation.
function relu_upper_bound(lower, upper)
    lower_r = clamp.(lower, -Inf, 0)
    upper_r = clamp.(upper, 0, Inf)
    #lower_r .= min.(lower_r, 0)
    upper_r .= max.(upper_r, lower_r .+ 1e-8)
    upper_slope = upper_r ./ (upper_r .- lower_r) #the slope of the relu upper bound
    upper_bias = - lower_r .* upper_slope #the bias of the relu upper bound
    return upper_slope, upper_bias
end


function clamp_mutiply_A(last_A, slope_pos, slope_neg) 
    A_pos = clamp.(last_A, 0, Inf)
    A_neg = clamp.(last_A, -Inf, 0)
    slope_pos = repeat(reshape(slope_pos,(1, size(slope_pos)...)), size(A_pos)[1], 1, 1) #add spec dim for slope_pos
    slope_neg = repeat(reshape(slope_neg,(1, size(slope_neg)...)), size(A_neg)[1], 1, 1) #add spec dim for slope_pos
    New_A = slope_pos .* A_pos .+ slope_neg .* A_neg 
    return New_A
end 

#using last_A for getting New_A
function multiply_by_A_signs(last_A, slope_pos, slope_neg)
    if ndims(slope_pos) == 1
        # Special case for LSTM, the bias term is 1-dimension. 
        New_A = clamp.(last_A, 0, Inf) .* slope_pos .+ clamp.(last_A, -Inf, 0) .* slope_neg
    else
        New_A = clamp_mutiply_A(last_A, slope_pos, slope_neg)
        return New_A
    end
end


#bound oneside of the relu, like upper or lower
function bound_oneside(last_A, slope_pos, slope_neg)
    if isnothing(last_A)
        return nothing, nothing
    end
    New_A = multiply_by_A_signs(last_A, slope_pos, slope_neg)
    return New_A
end


alpha = [1]
unstable_mask = [2]
lower_mask = [3]
upper_slope = [1]

m = []
a = AlphaLayer(alpha, false, unstable_mask, lower_mask, upper_slope)
b = AlphaLayer([4], false, unstable_mask, lower_mask, upper_slope)
c = AlphaLayer([200], false, unstable_mask, lower_mask, upper_slope)
push!(m, a)
push!(m, b)
push!(m, c)
m = Chain(m)
println(b)
println(a([1]))
println(Flux.params(m))

AlphaLayer([4], false, [2], [3], [1])


[-1]


Params([[1], [4], [200]])


In [ ]:
for activation_node in model_info.activation_nodes
    batch_info[activation_node][:split_active] = []
end
primals, duals, mini_inp = None, None, None
upper_bound = zeros(size(lower_bound)) .+ Inf

In [45]:
function test_mlp(prop_method)
    small_nnet_file = "/home/verification/ModelVerification.jl/test/networks/small_nnet.nnet"
    # small_nnet encodes the simple function 24*max(x + 1.5, 0) + 18.5
    small_nnet = read_nnet(small_nnet_file, last_layer_activation = ModelVerification.ReLU())
    flux_model = Flux.Chain(small_nnet)
    #ONNXNaiveNASflux.save("/home/verification/ModelVerification.jl/small_nnet.onnx", flux_model, (1,1))
    # println(flux_model)
    # println(flux_model.layers[1].weight, " ", flux_model.layers[1].bias) # max(x+1.5, 0) max(x+1.5, 0)              [0,4]
    # println(flux_model.layers[2].weight, " ", flux_model.layers[2].bias) # 4*max(x+1.5, 0)+2.5 4*max(x+1.5, 0)+2.5  [2.5, 18.5]
    # println(flux_model.layers[3].weight, " ", flux_model.layers[3].bias) # 24*max(x+1.5, 0)+18.5                    [18.5, 114.5]
    in_hyper  = Hyperrectangle(low = [-2.5], high = [2.5]) # expected out: [18.5, 114.5]
    out_violated    = Hyperrectangle(low = [19], high = [114]) # 20.0 ≤ y ≤ 90.0
    out_holds = Hyperrectangle(low = [18], high = [115.0]) # -1.0 ≤ y ≤ 50.0
    comp_violated    = Complement(Hyperrectangle(low = [10], high = [19])) # y ≤ 10.0 or 19 ≤ y
    comp_holds    = Complement(Hyperrectangle(low = [115], high = [118])) # y ≤ 10.0 or 18 ≤ y
    info = nothing
    search_method = BFS(max_iter=100, batch_size=10)
    split_method = Bisect(1)
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, out_violated)).status == :violated
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_violated)).status == :violated
    #= @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, out_violated)).status == :violated
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_holds)).status == :holds
    @test verify(search_method, split_method, prop_method, Problem(flux_model, in_hyper, comp_violated)).status == :violated =#
end
@timed begin
    #for i in 1:1
        test_mlp(AlphaCrown(Crown(true, true), true, true, Flux.Optimiser(Flux.ADAM(0.1)), 2))
        #test_mlp(Ai2z())
        #test_mlp(Crown(true, true))
        #test_mlp(StarSet(Crown(true, true)))
    #end
end

relu_3
dense_2
relu_2
dense_1
relu_1
dense_0
data_0
relu_3
dense_2
relu_2
dense_1
relu_1
dense_0
data_0


Error During Test at /home/verification/ModelVerification.jl/tmp/test.ipynb:22
  Test threw exception
  Expression: (verify(search_method, split_method, prop_method, Problem("/home/verification/ModelVerification.jl/small_nnet.onnx", in_hyper, comp_violated))).status == :violated
  DimensionMismatch: new dimensions (2,) must be consistent with array size 4
  Stacktrace:
   [1] (::Base.var"#throw_dmrsa#289")(dims::Tuple{Int64}, len::Int64)
     @ Base ./reshapedarray.jl:41
   [2] reshape
     @ ./reshapedarray.jl:45 [inlined]
   [3] reshape(parent::Array{Float64, 3}, dims::Int64)
     @ Base ./reshapedarray.jl:117
   [4] check_inclusion(prop_method::AlphaCrown, model::Chain{Tuple{Dense{typeof(relu), Matrix{Float64}, Vector{Float64}}, Dense{typeof(relu), Matrix{Float64}, Vector{Float64}}, Dense{typeof(relu), Matrix{Float64}, Vector{Float64}}}}, batch_input::Vector{Any}, bound::ModelVerification.AlphaCrownBound, batch_out_spec::ModelVerification.LinearSpec)
     @ ModelVerification ~/Model

Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

In [14]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(512, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
input_set = ImageConvexHull(image_seeds)
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
flux_model = model
image_shape = (32, 32, 3, 5)
println(image_seeds)

Array{Float32, 3}[[0.23137255

error in running finalizer: InterruptException()
jl_mutex_unlock at /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/src/julia_locks.h:129 [inlined]
ijl_iolock_end at /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/src/jl_uv.c:77
iolock_end at ./libuv.jl:49 [inlined]
uvfinalize at ./asyncevent.jl:188
unknown function (ip: 0x7ff85ff62212)
_jl_invoke at /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/src/gf.c:2377 [inlined]
ijl_apply_generic at /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/src/gf.c:2559
jl_apply at /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/src/julia.h:1843 [inlined]
run_finalizer at /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/src/gc.c:280
jl_gc_run_finalizers_in_list at /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/src/gc.c:367
run_finalizers at /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/src/gc.c:410
jl_gc_run_pending_finalizers at /cache/buil

 0.0627451 0.09803922 0.12941177 0.19607843 0.2784314 0.38039216 0.4509804 0.5372549 0.6039216 0.6039216 0.5686275 0.5568628 0.61960787 0.5686275 0.5803922 0.58431375 0.5764706 0.59607846 0.5686275 0.56078434 0.56078434 0.5529412 0.56078434 0.58431375 0.6745098 0.7921569 0.84705883 0.8627451 0.8156863 0.7058824 0.69411767; 0.16862746 0.0 0.0627451 0.14901961 0.23137255 0.32941177 0.43529412 0.46666667 0.5019608 0.6039216 0.60784316 0.57254905 0.5529412 0.6039216 0.58431375 0.57254905 0.56078434 0.5254902 0.45882353 0.49803922 0.49803922 0.49019608 0.5137255 0.54509807 0.52156866 0.5647059 0.73333335 0.75686276 0.7882353 0.7882353 0.6784314 0.65882355; 0.19607843 0.07058824 0.19215687 0.34117648 0.4 0.43137255 0.48235294 0.50980395 0.5176471 0.6117647 0.6117647 0.57254905 0.54901963 0.5568628 0.5764706 0.5686275 0.5647059 0.54901963 0.44705883 0.5019608 0.5058824 0.5137255 0.54509807 0.5411765 0.53333336 0.5294118 0.5921569 0.65882355 0.7294118 0.7764706 0.7294118 0.7019608; 0.26666668 

 0.21960784 0.21960784 0.38039216 0.5921569; 0.5803922 0.47843137 0.42745098 0.36862746 0.2627451 0.1254902 0.25882354 0.4 0.5254902 0.52156866 0.5254902 0.52156866 0.52156866 0.5411765 0.50980395 0.4862745 0.44705883 0.5019608 0.5921569 0.54901963 0.53333336 0.7019608 0.78039217 0.74509805 0.6039216 0.53333336 0.45882353 0.2784314 0.12941177 0.20784314 0.3254902 0.48235294;;; 0.24313726 0.078431375 0.09411765 0.09803922 0.1254902 0.1882353 0.27058825 0.32156864 0.39215687 0.47058824 0.47843137 0.44705883 0.4509804 0.5137255 0.4509804 0.45490196 0.4509804 0.43529412 0.44705883 0.4117647 0.40784314 0.40784314 0.4 0.40392157 0.41960785 0.5019608 0.6156863 0.68235296 0.7137255 0.6666667 0.54509807 0.5647059; 0.18039216 0.0 0.02745098 0.078431375 0.1254902 0.20784314 0.29411766 0.29803923 0.32156864 0.4392157 0.45882353 0.42745098 0.41568628 0.46666667 0.42745098 0.39215687 0.37254903 0.34509805 0.29411766 0.32156864 0.3137255 0.29803923 0.3137255 0.34117648 0.3137255 0.34509805 0.5058824 

0.37254903 0.3529412 0.35686275 0.38431373 0.38039216 0.35686275 0.45490196 0.48235294 0.3372549 0.25490198 0.36078432 0.45490196 0.47843137 0.44313726 0.4745098 0.62352943 0.7176471 0.6784314 0.5647059 0.52156866 0.5803922 0.72156864; 0.49019608 0.3254902 0.32941177 0.33333334 0.18431373 0.039215688 0.23921569 0.36078432 0.3372549 0.3372549 0.3372549 0.34117648 0.34901962 0.34901962 0.34901962 0.34509805 0.37254903 0.46666667 0.4117647 0.2901961 0.32156864 0.45490196 0.53333336 0.58431375 0.64705884 0.70980394 0.60784316 0.3647059 0.11372549 0.12156863 0.24313726 0.4627451; 0.4862745 0.34117648 0.28627452 0.24313726 0.16470589 0.050980393 0.18431373 0.2901961 0.37254903 0.36078432 0.3647059 0.36078432 0.35686275 0.38039216 0.36078432 0.34509805 0.32156864 0.4 0.5019608 0.4745098 0.46666667 0.63529414 0.72156864 0.68235296 0.5176471 0.41960785 0.3372549 0.1882353 0.07450981 0.13333334 0.20784314 0.36078432;;; 0.24705882 0.078431375 0.08235294 0.06666667 0.08235294 0.11372549 0.15686275

 0.24313726 0.22745098 0.24313726 0.13333334 0.050980393 0.0627451 0.13333334 0.12156863 0.05490196 0.078431375 0.12156863 0.15294118 0.18039216 0.22352941 0.18039216 0.043137256 0.0 0.22352941; 0.40784314 0.23529412 0.22745098 0.19607843 0.1254902 0.09411765 0.11372549 0.13333334 0.1882353 0.20392157 0.22352941 0.22352941 0.22352941 0.21960784 0.23529412 0.20392157 0.21568628 0.12941177 0.050980393 0.05490196 0.16862746 0.13725491 0.03137255 0.015686275 0.09411765 0.17254902 0.27450982 0.4 0.38431373 0.23529412 0.22352941 0.40784314; 0.42352942 0.21568628 0.19607843 0.19607843 0.105882354 0.05882353 0.14509805 0.12941177 0.16862746 0.20392157 0.21568628 0.2 0.20392157 0.23529412 0.23529412 0.19215687 0.18039216 0.14901961 0.0627451 0.05882353 0.19607843 0.15294118 0.039215688 0.03529412 0.16470589 0.34117648 0.41568628 0.4117647 0.34117648 0.27450982 0.36862746 0.54901963; 0.4 0.19607843 0.19607843 0.1882353 0.09019608 0.0 0.14901961 0.23137255 0.1764706 0.1764706 0.18039216 0.1803921

0.36862746 0.4627451 0.4 0.23137255 0.3529412; 0.21960784 0.25490198 0.25490198 0.27058825 0.3529412 0.34117648 0.3372549 0.38039216 0.32941177 0.27058825 0.3764706 0.6862745 0.72156864 0.70980394 0.78431374 0.78431374 0.7921569 0.8235294 0.8156863 0.81960785 0.8 0.80784315 0.5882353 0.1882353 0.20392157 0.2509804 0.2784314 0.28235295 0.3372549 0.21568628 0.19215687 0.45490196; 0.3019608 0.32941177 0.3254902 0.38039216 0.36862746 0.38431373 0.3372549 0.28627452 0.25490198 0.29411766 0.28235295 0.48235294 0.7254902 0.7058824 0.7372549 0.70980394 0.65882355 0.7019608 0.75686276 0.84705883 0.827451 0.8352941 0.64705884 0.20784314 0.1254902 0.19215687 0.16862746 0.16862746 0.33333334 0.1254902 0.21176471 0.5254902; 0.36862746 0.36078432 0.3529412 0.34509805 0.27058825 0.29803923 0.3529412 0.30980393 0.21960784 0.3137255 0.4 0.43529412 0.57254905 0.7607843 0.7176471 0.7254902 0.5372549 0.53333336 0.654902 0.7882353 0.8156863 0.827451 0.68235296 0.21960784 0.09411765 0.09803922 0.09803922 0.

 0.7921569 0.7764706 0.84313726 0.9019608 0.8745098 0.85882354 0.8784314 0.5372549 0.23529412 0.78039217 0.54901963 0.039215688 0.07450981 0.09803922 0.19607843 0.5294118 0.49411765 0.42745098 0.3254902 0.14509805; 0.64705884 0.92156863 0.92156863 0.7647059 0.827451 0.9411765 0.9254902 0.79607844 0.7490196 0.8980392 0.8980392 0.9019608 0.8980392 0.8901961 0.90588236 0.90588236 0.8784314 0.85882354 0.87058824 0.6509804 0.29411766 0.6509804 0.627451 0.23137255 0.08235294 0.08235294 0.09411765 0.16862746 0.19215687 0.27058825 0.24313726 0.078431375; 0.627451 0.8901961 0.9529412 0.78431374 0.7607843 0.9411765 0.95686275 0.9411765 0.92941177 0.92941177 0.92156863 0.9098039 0.8980392 0.8980392 0.9019608 0.89411765 0.88235295 0.8666667 0.8745098 0.78431374 0.62352943 0.6901961 0.7058824 0.6509804 0.5019608 0.25490198 0.18039216 0.18039216 0.17254902 0.23529412 0.18431373 0.023529412; 0.45882353 0.56078434 0.69411767 0.7176471 0.5254902 0.6666667 0.72156864 0.81960785 0.79607844 0.73333335 0.7

0.2901961 0.31764707 0.29411766 0.08235294; 0.827451 0.9372549 0.8901961 0.8862745 0.9137255 0.9098039 0.70980394 0.5686275 0.46666667 0.36078432 0.40392157 0.47058824 0.5058824 0.46666667 0.6901961 0.90588236 0.84705883 0.85882354 0.8784314 0.44705883 0.33333334 0.84313726 0.63529414 0.1882353 0.19607843 0.22745098 0.25490198 0.42745098 0.5176471 0.4 0.23921569 0.16470589; 0.63529414 0.92156863 0.87058824 0.8117647 0.9137255 0.9254902 0.85882354 0.63529414 0.7294118 0.79607844 0.7921569 0.8039216 0.8039216 0.78039217 0.8352941 0.8862745 0.8627451 0.8666667 0.9019608 0.5647059 0.25490198 0.77254903 0.53333336 0.02745098 0.06666667 0.105882354 0.22352941 0.5803922 0.54901963 0.46666667 0.36862746 0.1882353; 0.6117647 0.8980392 0.9098039 0.76862746 0.8352941 0.92941177 0.91764706 0.79607844 0.7607843 0.92156863 0.91764706 0.8784314 0.8627451 0.8666667 0.8901961 0.8980392 0.8784314 0.8666667 0.8784314 0.65882355 0.3019608 0.67058825 0.6392157 0.22745098 0.07058824 0.078431375 0.11372549 0

In [23]:
prop_method = ImageStar()
@timed verify(search_method, split_method, prop_method, Problem(flux_model, input_set, output_set))

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, image_seeds, output_set))

In [ ]:
model = Chain([
    Flux.flatten,
    Dense(784, 200, relu),
    Dense(200, 10)
])
image_seeds = [MNIST(:train)[i].features for i in 1:1]
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/debug.onnx"
Flux_model = model
image_shape = (28, 28, 1, 1)

In [ ]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    Conv((3, 3), 8 => 8, relu, pad=SamePad(), stride=(2, 2)),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(128, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)
onnx_model_path = "/home/verification/ModelVerification.jl/mlp.onnx"
Flux_model = model
image_shape = (32, 32, 3, 2)

In [ ]:
prop_method = ImageStar()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_shape, image_seeds, output_set))

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(onnx_model_path, Flux_model, image_shape, image_seeds, output_set))

In [ ]:
model = Chain([
    Conv((3, 3), 3 => 128, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(128),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 128 => 128, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    ConvTranspose((3, 3), 128 => 128, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(32768, 100, relu),
    Dense(100, 10)
])
testmode!(model)
# image_seeds = CIFAR10(:train)[1:5].features # 32 x 32 x 3 x 5
image_seeds = [CIFAR10(:train)[i].features for i in 1:2]
# println(typeof(image_seeds[1][1,1,1,1]))
search_method = BFS(max_iter=1, batch_size=1)
split_method = Bisect(1)
output_set = BallInf(zeros(10), 1.0)

In [ ]:
prop_method = ImageStarZono()
@timed verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

In [ ]:
Profile.clear()
@profile verify(search_method, split_method, prop_method, Problem(model, image_seeds, output_set))

In [ ]:
open("./prof.txt", "w") do s
    Profile.print(IOContext(s, :displaysize => (24, 500)))
end